In [4]:
import os
from azure.servicebus import ServiceBusService, Message, Topic, Rule, DEFAULT_RULE_NAME
from firebase import firebase
import json
import uuid

ns_key = os.environ.get('NS_KEY')
firebase = firebase.FirebaseApplication('https://logbasedev.firebaseio.com/', None)

bus_service = ServiceBusService(
    service_namespace='onroad-ns',
    shared_access_key_name='RootManageSharedAccessKey',
    shared_access_key_value=ns_key)


In [19]:
alerts = firebase.get('/account/simplelogin:2/alerts', None)
alertCount =  len(alerts)
while True:
    msg = bus_service.receive_subscription_message('onroad-alerts', 'alerts-consumer', peek_lock=False)
    if msg.body:
        alert = json.loads(msg.body)
        print('running... ')
        #TODO: Improve, take the last item only  
        for event in alert['location']:
            alert_location = {'latitude':event['lat'], 'longitude':event['long'], 'locationtime':event['time']};
        #TODO: Remove device number hard coding
        new_alert = {'alertid': str(uuid.uuid1()), 'alerttype': alert['alert'], 'devicenumber': '8650670123456', 'latitude': alert_location['latitude'], 'longitude': alert_location['longitude'], 'status':'Open', 'time': alert_location['locationtime']}
        print new_alert
        alertCount = alertCount + 1
        #firebase.patch('/account/simplelogin:2/alerts/' + str(alertCount), new_alert)

running... 
{'status': 'Open', 'alerttype': u'unplugged', 'alertid': '2cbb6acc-20aa-11e5-9fc3-b8e85647c78a', 'longitude': 77.00254386, 'devicenumber': '8650670123456', 'time': 1435552978011, 'latitude': 11.01444401}
running... 
{'status': 'Open', 'alerttype': u'panic', 'alertid': '374e3154-20aa-11e5-ace0-b8e85647c78a', 'longitude': 77.00254386, 'devicenumber': '8650670123456', 'time': 1435552978011, 'latitude': 11.01444401}
running... 
{'status': 'Open', 'alerttype': u'panic', 'alertid': '41fea391-20aa-11e5-8e72-b8e85647c78a', 'longitude': 77.00254386, 'devicenumber': '8650670123456', 'time': 1435552978011, 'latitude': 11.01444401}


KeyboardInterrupt: 